# ツイート内容のカテゴリ分類
- 「みずほ」を含むツイートの中を、みずほ銀行関連、みずほ銀行アプリ関連のツイートを自動で分類分けする
    - みずほ：人
    - みずほ：口座売買
    - みずほ：売買
    - みずほ：投資関連
    - みずほ：就活関連
    - `みずほ：みずほ銀行関連 ★`
    - `みずほ：アプリ関連 ★`
    - みずほ：その他、みずほ銀行に関係ないもの

In [2]:
from IPython.display import clear_output
import pandas as pd
import re
import numpy as np

In [3]:
# !pip install fugashi ipadic transformers==4.9.2 tensorflow-addons==0.13.0
!pip install ipadic fugashi transformers tensorflow-addons

from transformers import TFBertForSequenceClassification, BertJapaneseTokenizer
import tensorflow as tf

import tensorflow_addons as tfa
from matplotlib import pyplot as plt

model_name = "cl-tohoku/bert-base-japanese"
model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=7)
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

clear_output()

In [5]:
!pip install neologdn
import neologdn

!pip install emoji
import emoji

!pip install demoji
import demoji

clear_output()

In [ ]:
#Drive環境のマウント
from google.colab import drive
drive.mount('/content/drive')

## データ
- TweeterのAPIで「みずほ -みずほ町 -みずほちゃん」のクエリで取得したデータ
- categoryについては、textを見て自分で振った、ツイート内容のカテゴリ


### データ取得ソースの一部
```python
# -----
def get_tweets_and_save(search_word, max_search_num=100, csv_name="tweet_data.csv"):
    def write_csv(df, csv_name="tweet_data.csv"):
        prev_df = pd.read_csv(output_path+csv_name)
        df = pd.concat([prev_df, df])
        df = df.reset_index(drop=True)
        df = df[~df["text"].duplicated()]
        df.to_csv(output_path+csv_name, index=False)
    
    # jsonの展開
    def add_public_metrics_column(df):
        df = df.reset_index(drop=True)
        public_metrics = np.array(df["public_metrics"])
        # public_metrics = np.array([s.replace("'", '"') for s in public_metrics])

        df["retweet_count"] = 0
        df["reply_count"] = 0
        df["like_count"] = 0
        df["quote_count"] = 0
        
        for i, d in enumerate(public_metrics):
            # d = json.loads(d)
            for key, value in d.items():
                df.at[i, key] = value

        return df.drop("public_metrics", axis=1)
    
    
    # 最新のツイートを取得
    tweets = client.search_recent_tweets(
        query=search_word, 
        max_results=max_search_num,
        tweet_fields = ['author_id', 'created_at', 'public_metrics'],
        user_fields = 'profile_image_url',
        expansions = ['author_id', 'attachments.media_keys'],
        # media_fields = 'url',
    )

    # df化
    df = pd.DataFrame(tweets[0])
    
    # jsonの展開
    df = add_public_metrics_column(df)
    
    # 保存
    write_csv(df, csv_name)

# -----

while True:
    # ----- みずほ
    try:
        get_tweets_and_save("みずほ -みずほ町 -みずほちゃん", max_search_num, "tweet_data.csv")
        
    except Exception as e:
        print(e)
        print("    retry -> ", end="")
        
        # retry
        try:
            get_tweets_and_save("みずほ -みずほ町 -みずほちゃん", max_search_num, "tweet_data.csv")
            print("successfully")
            
        except Exception as e:
            print("failed")
            print(e)
    # -----
            
    
    # ----- 銀行 → 今回は使用していないデータ
    try:
        get_tweets_and_save("銀行", max_search_num, "tweet_data_bank.csv")
        
    except Exception as e:
        print(e)
        print("    retry -> ", end="")
        
        # retry
        try:
            get_tweets_and_save("銀行", max_search_num, "tweet_data_bank.csv")
            print("successfully")
            
        except Exception as e:
            print("failed")
            print(e)
    # -----
    
    # 15分おき
    time.sleep(60*15)

```


### カテゴリ値について
```json
category = {
    0: "売買", 
    1: "口座売買", 
    2: "人", 
    3: "みずほ銀行関連", 
    4: "みずほアプリ関連", 
    5: "その他関係があまりないもの", 
    6: "投資関連", 
}
```


## データの読み込み

In [7]:
#データのインポート(自身のディレクトリに合わせて変更してください)
data = pd.read_csv("/content/drive/MyDrive/データ分析/dl4us/final_report/input/tweet_data_labeled_v2.csv")

In [8]:
train = pd.read_csv("/content/drive/MyDrive/データ分析/dl4us/final_report/output/train.csv") 
test = pd.read_csv("/content/drive/MyDrive/データ分析/dl4us/final_report/output/test.csv") 

In [9]:
train.head()

,Unnamed: 0,attachments,author_id,created_at,id,text,category,retweet_count,reply_count,like_count,quote_count,edit_history_tweet_ids
0,0,NaN,1.475759e+18,2022-09-04 16:08:15+00:00,1.566458e+18,RT @s8mrs: 【買取】ハイキュー クロニクル フォトカード フォトカ 選手名鑑カード...,0.0,17.0,0.0,0.0,0.0,NaN
1,1,NaN,1.565875e+18,2022-09-04 16:07:21+00:00,1.566458e+18,困ってませんか⁉️\n\n騙される前に🙇‍♂️\n\n口座買取3年の実績🌸\n\n犯罪には絶...,1.0,0.0,0.0,0.0,0.0,NaN
2,2,NaN,1.456662e+18,2022-09-04 16:06:48+00:00,1.566458e+18,@kuroshio_46 みずほの地元はイナイレ映らんもんな、、,2.0,0.0,0.0,0.0,0.0,NaN
3,3,NaN,1.565875e+18,2022-09-04 16:05:56+00:00,1.566458e+18,困ってませんか⁉️\n\n騙される前に🙇‍♂️\n\n口座買取3年の実績🌸\n\n犯罪には絶...,1.0,0.0,0.0,0.0,0.0,NaN
4,4,NaN,2.860389e+09,2022-09-04 16:05:23+00:00,1.566457e+18,RT @ProfShimada: 文字通りのテロ犯罪カルト集団・北朝鮮と「接点」のある政治家...,2.0,700.0,0.0,0.0,0.0,NaN


In [10]:
test.head()

,Unnamed: 0,attachments,author_id,created_at,id,text,category,retweet_count,reply_count,like_count,quote_count,edit_history_tweet_ids
0,66,NaN,1.278201e+09,2022-09-04 14:58:47+00:00,1.566441e+18,【ぶりっ子に多い名前】\n・わかな\n・のぞみ\n・えり\n・あやか\n・あゆみ\n・ゆみ\...,NaN,0.0,0.0,0.0,0.0,NaN
1,721,NaN,1.297351e+18,2022-09-05 04:40:28+00:00,1.566647e+18,RT @ratiumu2: 今月発売なのでぜひ！！！！\n「新米錬金術師の店舗経営」の作者、...,NaN,5.0,0.0,0.0,0.0,NaN
2,722,{'media_keys': ['3_1566647137772785664']},1.622221e+09,2022-09-05 04:39:19+00:00,1.566647e+18,みずほリースとＳＥＭＩＴＥＣの、リースを活用した設備投資について https://t.co/...,NaN,0.0,0.0,0.0,0.0,NaN
3,723,NaN,3.369541e+08,2022-09-05 04:39:02+00:00,1.566647e+18,今度はみずほを語るメールが来てるが取引がねーワケ,NaN,0.0,0.0,0.0,0.0,NaN
4,724,"{'media_keys': ['3_1566647003131822080', '3_15...",1.195500e+18,2022-09-05 04:38:49+00:00,1.566647e+18,『ゆうた君代理\n【みずほ先生応援隊】\n医療班理学療法組(I.K)』\n\n昨日は[ゆうた...,NaN,0.0,1.0,1.0,0.0,NaN


## データの前処理
### 取り除くもの
- 改行文字
- 「RT xxx :」
- URL
- 絵文字

### 置換
- 数字：全て0に置き換え
- 大文字：小文字に置き換え
- カタカナ：全角カタカナに統一
- 半角記号：半角空白に置き換え
- 全角記号：半角空白に置き換え

### その他
- 重複するテキストがあれば取り除く

In [12]:
def get_text_data(df):
    df["text"] = df["text"].astype(str)

    # 改行文字などを置き換える
    def remove_escape_str(t):
        return t.replace('\n',' ').replace('\r',' ').replace("&nbsp", " ")
    
    # RT xxx :をから文字に置き換える
    def remove_RT(t):
        return re.sub("rt.+:", "", t)
    
    # URLを取り除く
    def remove_URL(t):
        return re.sub(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-]+', '', t)
    
    # 絵文字を取り除く
    def remove_emoji(t):
        return demoji.replace(string=t, repl='')
    
    # 数字を全て0に置き換える
    def replace_num(t):
        tmp = re.sub(r'(\d)([,.])(\d+)', r'\1\3', t)
        return re.sub(r'\d+', '0', tmp) 
    
    # 記号を置き換える
    def replace_symbol(t):
        # 半角記号の置換
        tmp = re.sub(r'[!-/:-@[-`{-~]', r' ', t)

        # 全角記号の置換 (ここでは0x25A0 - 0x266Fのブロックのみを除去)
        return re.sub(u'[■-♯]', ' ', tmp)

    # 大文字小文字を統一
    df["text"] = df["text"].map(lambda d: d.lower())

    # 文章のノーマライズ
    df["text"] = df["text"].map(lambda d: neologdn.normalize(d))

    # 改行文字などを置き換える
    df["text"] = df["text"].map(lambda d: remove_escape_str(d))

    # 先頭のRTの情報を消す
    df["text"] = df["text"].map(lambda d: remove_RT(d))

    # URLの情報を消す
    df["text"] = df["text"].map(lambda d: remove_URL(d))

    # 絵文字を取り除く
    df["text"] = df["text"].map(lambda d: remove_emoji(d))

    # 数字を全て0に置き換える
    df["text"] = df["text"].map(lambda d: replace_num(d))

    # 記号を置き換える
    df["text"] = df["text"].map(lambda d: replace_symbol(d))

    # 重複するテキストを取り除く
    df = df.drop_duplicates(subset=['text'])
    
    return df

In [13]:
train = get_text_data(train)
test = get_text_data(test)

clear_output()

In [14]:
display(train["category"].value_counts())

train.loc[train["category"] == 5.0, "category"] = 3
train.loc[train["category"] == 22.0, "category"] = 2
train.loc[train["category"] == -1.0, "category"] = 5

 3.0     359
 2.0     189
-1.0      92
 4.0      79
 1.0      57
 6.0      44
 0.0      30
 5.0       8
 22.0      1
Name: category, dtype: int64

##　モデルに入れる形式に変換する
- 形態素解析
- id変換
- など

In [15]:
def to_features(texts, max_length):
    shape = (len(texts), max_length)
    print(shape)

    input_ids = np.zeros(shape, dtype="int32")
    attention_mask = np.zeros(shape, dtype="int32")
    token_type_ids = np.zeros(shape, dtype="int32")
    
    for i, text in enumerate(texts):
        encoded_dict = tokenizer.encode_plus(text, max_length=max_length, pad_to_max_length="longest", truncation=True)

        input_ids[i] = encoded_dict["input_ids"]
        attention_mask[i] = encoded_dict["attention_mask"]
        token_type_ids[i] = encoded_dict["token_type_ids"]
        
    return [input_ids, attention_mask, token_type_ids]

In [ ]:
train_texts = np.array(train["text"])
np.random.seed(42)
np.random.shuffle(train_texts)

train_y = np.array(train["category"])
np.random.seed(42)
np.random.shuffle(train_y)

test_texts = np.array(test["text"])

train_features = to_features(train_texts, max_length=100)
test_features = to_features(test_texts, max_length=100)

## bertモデルの定義と学習
- 最終層から10層前まで学習する

In [21]:
# BERTの最終層から10個前までのの層のみを学習する
for layer in model.bert.encoder.layer[:-10]:
    layer.trainable = False
                         
# BERT部分と分類器で学習率を変える
optimizers = [
    tf.keras.optimizers.Adam(learning_rate=1e-5),
    tf.keras.optimizers.Adam(learning_rate=1e-4)
]
optimizers_and_layers = [
    (optimizers[0], model.bert.encoder.layer[:-1]),
    (optimizers[1], model.classifier)
]
optimizer = tfa.optimizers.MultiOptimizer(optimizers_and_layers)

In [22]:
model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, verbose=1, restore_best_weights=True
)

model.fit(
    train_features,
    train_y,
    batch_size=32,
    validation_split=0.2,
    epochs=50,
    callbacks=[early_stopping]
)

# model.save_pretrained("./trained_BERT")

Epoch 1/50
22/22 [==============================] - 43s 865ms/step - loss: 1.7169 - sparse_categorical_accuracy: 0.3668 - val_loss: 1.6456 - val_sparse_categorical_accuracy: 0.3488
Epoch 2/50
22/22 [==============================] - 13s 587ms/step - loss: 1.4208 - sparse_categorical_accuracy: 0.4833 - val_loss: 1.4345 - val_sparse_categorical_accuracy: 0.4186
Epoch 3/50
22/22 [==============================] - 13s 590ms/step - loss: 1.2273 - sparse_categorical_accuracy: 0.5852 - val_loss: 1.1499 - val_sparse_categorical_accuracy: 0.5988
Epoch 4/50
22/22 [==============================] - 13s 592ms/step - loss: 1.0188 - sparse_categorical_accuracy: 0.6579 - val_loss: 1.0057 - val_sparse_categorical_accuracy: 0.6221
Epoch 5/50
22/22 [==============================] - 13s 605ms/step - loss: 0.8892 - sparse_categorical_accuracy: 0.6783 - val_loss: 0.9047 - val_sparse_categorical_accuracy: 0.6512
Epoch 6/50
22/22 [==============================] - 13s 603ms/step - loss: 0.7559 - sparse_cate

## 予測の実施

In [23]:
import gc
class gc_collect(tf.keras.callbacks.Callback):
  def on_predict_batch_begin(self, batch, logs=None):
      gc.collect()

pred_y = model.predict(
    test_features, 
    # batch_size=8, 
    callbacks=[gc_collect()]
    )

pred_y = pred_y["logits"]
pred_y_arg_max = np.argmax(pred_y, axis=1)

# print(classification_report(test_y, pred_y, digits=3))
# print(f"accuracy: {accuracy_score(test_y, pred_y):.4f}")

   6/1331 [..............................] - ETA: 10:24

1331/1331 [==============================] - 621s 464ms/step


In [24]:
pred = test.copy()
pred["pred"] = pred_y_arg_max
pred.to_csv("/content/drive/MyDrive/データ分析/dl4us/final_report/output/predict.csv")

# Shift-jisエンコードでもファイルを保存(Excelで見れるようにするため)
# 一度ファイルオブジェクトをエラー無視、書き込みで開く
with open("/content/drive/MyDrive/データ分析/dl4us/final_report/output/predict_shift_jis.csv", mode="w", encoding="shift-jis", errors="ignore") as f:
    # pandasでファイルオブジェクトに書き込む
    pred.to_csv(f)

In [25]:
pred.head()

,Unnamed: 0,attachments,author_id,created_at,id,text,category,retweet_count,reply_count,like_count,quote_count,edit_history_tweet_ids,pred
0,66,NaN,1.278201e+09,2022-09-04 14:58:47+00:00,1.566441e+18,【ぶりっ子に多い名前】 ・わかな ・のぞみ ・えり ・あやか ・あゆみ ・ゆみ ・ちなつ ・...,NaN,0.0,0.0,0.0,0.0,NaN,2
1,721,NaN,1.297351e+18,2022-09-05 04:40:28+00:00,1.566647e+18,今月発売なのでぜひ 「新米錬金術師の店舗経営」の作者、いつきみずほさんからありがたい...,NaN,5.0,0.0,0.0,0.0,NaN,2
2,722,{'media_keys': ['3_1566647137772785664']},1.622221e+09,2022-09-05 04:39:19+00:00,1.566647e+18,みずほリースとsemitecの、リースを活用した設備投資について,NaN,0.0,0.0,0.0,0.0,NaN,6
3,723,NaN,3.369541e+08,2022-09-05 04:39:02+00:00,1.566647e+18,今度はみずほを語るメールが来てるが取引がねーワケ,NaN,0.0,0.0,0.0,0.0,NaN,3
4,724,"{'media_keys': ['3_1566647003131822080', '3_15...",1.195500e+18,2022-09-05 04:38:49+00:00,1.566647e+18,『ゆうた君代理 【みずほ先生応援隊】 医療班理学療法組 i k 』 昨日は ゆうた君 の心...,NaN,0.0,1.0,1.0,0.0,NaN,2
